In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf

In [31]:
path=r"C:\Users\great\OneDrive\Desktop\Ravdess"
k=0
xg_data=np.ndarray(shape=(24*60,120000))
yg_data=np.ndarray(shape=(24*60,1))
for i,actor in enumerate(os.listdir(path)):
    path1 = os.path.join(path,actor)
    for j, file in enumerate(os.listdir(path1)):
        if file=='test':
            continue
        b = file[18:20]
        yg_data[k,0]= int(b) % 2
        temp_path = os.path.join(path1, file)
        # print(temp_path)
        var, _ = librosa.load(temp_path)
        xg_data[k, :var.shape[0]] = var[:]
        xg_data[k,var.shape[0]:] = 0
        k = k + 1

In [9]:
def extract_feature(file):
    sr=22100
    result=np.array([])
    mfcc=np.mean(librosa.feature.mfcc(y=file,sr=sr,n_mfcc=40).T,axis=0)
    result=np.hstack((result,mfcc))
    chroma=np.mean(librosa.feature.chroma_stft(y=file,sr=sr).T,axis=0)
    result=np.hstack((result,chroma))
    mel=np.mean(librosa.feature.melspectrogram(y=file,sr=sr).T,axis=0)
    result=np.hstack((result,mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=file, sr=sr).T,axis=0)
    result=np.hstack((result,spectral_rolloff))
    FRAME_SIZE = 1024
    HOP_LENGTH = 512
    rms = librosa.feature.rms(y=file, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
    zcr_voice = librosa.feature.zero_crossing_rate(y=file, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
    return result

In [32]:
def build_fc_model_gender(input_shape=(180,1), num_class=2):


    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',input_shape=input_shape,kernel_regularizer=tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(64,activation='relu'),
                                    tf.keras.layers.Dense(num_class,activation='softmax')
                                    ])

    return model
model_gender=build_fc_model_gender()

In [33]:
xg=[]
for i in range(24*60):
    feature=extract_feature(xg_data[i,:])
    xg.append(feature)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train, x_test,y_train,y_test = train_test_split(np.array(xg), yg_data, test_size=0.2, random_state=0)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [41]:
from tensorflow.keras.callbacks import CSVLogger
model_gender.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
csv_logger = CSVLogger('model_history.log', separator=',', append=False)
history=model_gender.fit(x_train,y_train,epochs=500,batch_size=256,validation_data=(x_test,y_test),callbacks=[])

Epoch 1/500
5/5 [==============================] - 2s 65ms/step - loss: 5.9979 - accuracy: 0.5373 - val_loss: 2.4905 - val_accuracy: 0.9896
Epoch 2/500
5/5 [==============================] - 0s 25ms/step - loss: 2.6437 - accuracy: 0.8516 - val_loss: 2.4845 - val_accuracy: 0.8958
Epoch 3/500
5/5 [==============================] - 0s 23ms/step - loss: 2.4641 - accuracy: 0.8984 - val_loss: 2.3228 - val_accuracy: 0.9479
Epoch 4/500
5/5 [==============================] - 0s 21ms/step - loss: 2.2603 - accuracy: 0.9436 - val_loss: 2.0986 - val_accuracy: 0.9896
Epoch 5/500
5/5 [==============================] - 0s 33ms/step - loss: 2.0762 - accuracy: 0.9601 - val_loss: 1.9450 - val_accuracy: 0.9861
Epoch 6/500
5/5 [==============================] - 0s 22ms/step - loss: 1.9317 - accuracy: 0.9670 - val_loss: 1.8141 - val_accuracy: 0.9826
Epoch 7/500
5/5 [==============================] - 0s 21ms/step - loss: 1.7808 - accuracy: 0.9792 - val_loss: 1.6820 - val_accuracy: 0.9896
Epoch 8/500
5/5 [===

In [42]:
model_gender.evaluate(x_test,y_test)

9/9 [==============================] - 0s 3ms/step - loss: 0.0211 - accuracy: 0.9931


[0.021092262119054794, 0.9930555820465088]